In [4]:
#import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error



### deribit API Request
#### ALL Options data 

In [5]:
""" Download CC-Option Data from Deribit via public API """

"""
Matteo Bottacini -- matteo.bottacini@usi.ch
"""

# import modules
import json
import requests
from tqdm import tqdm
import sqlite3
import datetime


# functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1.  list of traded options for the selected coin;
                        2.  list of settlement period for the selected coin.
    """

    # requests public API
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)

    # get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period


def get_option_data(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas data frame with all option data for a given coin
    """

    # get option name and settlement
    coin_name = get_option_name_and_settlement(coin)[0]
    settlement_period = get_option_name_and_settlement(coin)[1]

    # initialize data frame
    coin_df = []

    # initialize progress bar
    pbar = tqdm(total=len(coin_name))

    # loop to download data for each Option Name
    for i in range(len(coin_name)):
        # download option data -- requests and convert json to pandas
        r = requests.get('https://test.deribit.com/api/v2/public/get_order_book?instrument_name=' + coin_name[i])
        result = json.loads(r.text)
        df = pd.json_normalize(result['result'])

        # add settlement period
        df['settlement_period'] = settlement_period[i]

        # append data to data frame
        coin_df.append(df)

        # update progress bar
        pbar.update(1)

    # finalize data frame
    coin_df = pd.concat(coin_df)

    # remove useless columns from coin_df
    columns = ['state', 'estimated_delivery_price']
    coin_df.drop(columns, inplace=True, axis=1)

    # close the progress bar
    pbar.close()

    return coin_df


### Specfic Settlement Period

In [44]:
import pandas as pd
import json
import requests
from tqdm import tqdm
import datetime

# functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1.  list of traded options for the selected coin;
                        2.  list of settlement period for the selected coin.
    """

    # requests public API
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)

    # get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period


def get_option_data(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas data frame with all option data for a given coin, filtered for options with settlement in months
    """

    # get option name and settlement
    coin_name, settlement_period = get_option_name_and_settlement(coin)

    # initialize data frame
    coin_df = []

    # initialize progress bar
    pbar = tqdm(total=len(coin_name))

    # loop to download data for each Option Name
    for i in range(len(coin_name)):
        # check if the settlement period is 'month'
        if settlement_period[i] != 'day':
            continue

        # download option data -- requests and convert json to pandas
        r = requests.get('https://test.deribit.com/api/v2/public/get_order_book?instrument_name=' + coin_name[i])
        result = json.loads(r.text)
        df = pd.json_normalize(result['result'])

        # add settlement period
        df['settlement_period'] = settlement_period[i]

        # append data to data frame
        coin_df.append(df)

        # update progress bar
        pbar.update(1)

    # finalize data frame
    if len(coin_df) > 0:
        coin_df = pd.concat(coin_df)

    # remove useless columns from coin_df
    columns = ['state', 'estimated_delivery_price']
    if not coin_df.empty:
        coin_df.drop(columns, inplace=True, axis=1)

    # close the progress bar
    pbar.close()

    return coin_df

# download data -- BTC and ETH Options
btc_data = get_option_data('BTC')

C:\Users\Ethan Vaz Falcao\AppData\Local\Temp\ipykernel_32604\2823738905.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coin_df = pd.concat(coin_df)
 12%|█▏        | 116/948 [00:58<07:00,  1.98it/s]


In [45]:
btc_data

,best_bid_amount,best_ask_amount,underlying_index,ask_iv,bid_iv,underlying_price,mark_iv,interest_rate,best_bid_price,best_ask_price,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,35.1,SYN.BTC-23OCT24,199.25,0.00,66993.4059,110.06,0.0,0.0000,0.0615,...,-17.85237,0.32441,0.00001,0.99240,0.00,0.0,NaN,NaN,NaN,day
0,7.8,0.0,SYN.BTC-23OCT24,0.00,110.08,66993.4059,110.06,0.0,0.0001,0.0000,...,-6.69872,0.46591,0.00002,-0.01078,185.58,10.1,-50.0000,0.0001,0.0003,day
0,0.0,35.1,SYN.BTC-23OCT24,169.86,0.00,66993.4059,94.87,0.0,0.0000,0.0470,...,-32.66551,0.68866,0.00003,0.98191,7249.60,2.2,0.0000,0.0490,0.0490,day
0,0.0,0.2,SYN.BTC-23OCT24,87.02,0.00,66991.6172,94.87,0.0,0.0000,0.0001,...,-12.08060,0.81839,0.00003,-0.02133,155.55,9.4,-75.0000,0.0001,0.0004,day
0,0.0,10.1,SYN.BTC-23OCT24,116.96,0.00,66991.6172,66.06,0.0,0.0000,0.0315,...,-28.55044,0.86435,0.00005,0.97634,76785.67,30.2,24.5902,0.0305,0.0390,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,0.0,SYN.BTC-24JAN25,0.00,0.00,68249.6938,79.10,0.0,0.0000,0.0000,...,-43.85890,103.88569,0.00001,-0.77427,0.00,0.0,NaN,NaN,NaN,day
0,46.7,0.1,SYN.BTC-24JAN25,91.88,48.76,68247.9308,80.34,0.0,0.0050,0.0544,...,-41.03940,95.70186,0.00001,0.19627,0.00,0.0,NaN,NaN,NaN,day
0,0.0,0.0,SYN.BTC-24JAN25,0.00,0.00,68247.9308,80.34,0.0,0.0000,0.0000,...,-41.03940,95.70186,0.00001,-0.80373,0.00,0.0,NaN,NaN,NaN,day
0,1.0,0.0,SYN.BTC-24JAN25,0.00,48.33,68245.6977,80.34,0.0,0.0030,0.0000,...,-36.97206,86.21703,0.00001,0.16618,0.00,0.0,NaN,NaN,NaN,day


In [40]:
import pandas as pd
import json
import asyncio
import aiohttp
from tqdm import tqdm
import datetime
import nest_asyncio

# Apply the patch to allow nested event loops
nest_asyncio.apply()

# asynchronous function to fetch data for each option name
async def fetch_option_data(session, coin_name):
    url = f'https://test.deribit.com/api/v2/public/get_order_book?instrument_name={coin_name}'
    async with session.get(url) as response:
        data = await response.json()

        # Check if 'result' key is present
        if 'result' in data:
            return data['result']
        else:
            print(f"Error: 'result' not found in response for {coin_name}. Response: {data}")
            return None

# Function to get the option names and settlement periods for the given coin
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1.  list of traded options for the selected coin;
                        2.  list of settlement period for the selected coin.
    """

    # requests public API
    r = requests.get(f"https://test.deribit.com/api/v2/public/get_instruments?currency={coin}&kind=option")
    result = json.loads(r.text)

    # get option name
    name = pd.json_normalize(result['result'])['instrument_name'].tolist()

    # get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period'].tolist()

    return name, settlement_period

# Main asynchronous function to get option data
async def fetch_all_options(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas data frame with all option data for a given coin, filtered for options with settlement in months
    """

    # get option name and settlement
    coin_name, settlement_period = get_option_name_and_settlement(coin)

    # initialize data frame
    coin_df = []

    # set up asynchronous session
    async with aiohttp.ClientSession() as session:
        # create a list of coroutines for each option name
        tasks = []
        for i in range(len(coin_name)):
            # check if the settlement period is 'day'
            if settlement_period[i] != 'day':  # Modify this as needed to filter different periods
                continue
            task = asyncio.ensure_future(fetch_option_data(session, coin_name[i]))
            tasks.append(task)

        # execute the tasks concurrently and gather results
        responses = await asyncio.gather(*tasks)

    # process the responses
    for i, result in enumerate(responses):
        if result is not None:
            df = pd.json_normalize(result)
            df['settlement_period'] = settlement_period[i]
            coin_df.append(df)

    # finalize data frame
    if len(coin_df) > 0:
        coin_df = pd.concat(coin_df)

    # remove useless columns from coin_df
    columns = ['state', 'estimated_delivery_price']
    if not coin_df.empty:
        coin_df.drop(columns, inplace=True, axis=1)

    return coin_df

# Wrapper function to handle the event loop and return the result
def get_option_data(coin):
    """
    Wrapper function to run the asynchronous fetch_all_options and return a pandas DataFrame.
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas DataFrame with the option data for the given coin
    """
    df = asyncio.run(fetch_all_options(coin))

    # Filter the DataFrame to only include rows where 'settlement_period' is 'day'
    filtered_df = df[df['settlement_period'] == 'day']

    return filtered_df


# Example usage:
# download data -- BTC and ETH Options
btc_data = get_option_data('BTC')
#eth_data = get_option_data('ETH')

# Now btc_data and eth_data are pandas DataFrames containing the options data.


Error: 'result' not found in response for BTC-24OCT24-69000-P. Response: {'jsonrpc': '2.0', 'error': {'message': 'too_many_requests', 'code': 10028}, 'testnet': True, 'usIn': 1729653525144330, 'usOut': 1729653525144354, 'usDiff': 24}
Error: 'result' not found in response for BTC-24OCT24-70000-P. Response: {'jsonrpc': '2.0', 'error': {'message': 'too_many_requests', 'code': 10028}, 'testnet': True, 'usIn': 1729653525144140, 'usOut': 1729653525144176, 'usDiff': 36}
Error: 'result' not found in response for BTC-24OCT24-74000-C. Response: {'jsonrpc': '2.0', 'error': {'message': 'too_many_requests', 'code': 10028}, 'testnet': True, 'usIn': 1729653525144183, 'usOut': 1729653525144217, 'usDiff': 34}


C:\Users\Ethan Vaz Falcao\AppData\Local\Temp\ipykernel_32604\227790089.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coin_df = pd.concat(coin_df)


In [41]:
 btc_data

,best_bid_amount,best_ask_amount,underlying_index,ask_iv,bid_iv,underlying_price,mark_iv,interest_rate,best_bid_price,best_ask_price,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,35.1,SYN.BTC-23OCT24,195.50,0.00,67004.0162,110.38,0.0,0.0000,0.0615,...,-17.93760,0.32502,0.00001,0.99239,0.00,0.0,NaN,NaN,NaN,day
0,7.8,0.0,SYN.BTC-23OCT24,0.00,110.32,67004.0162,110.38,0.0,0.0001,0.0000,...,-6.81608,0.46669,0.00002,-0.01080,185.58,10.1,-50.0000,0.0001,0.0003,day
0,0.0,35.1,SYN.BTC-23OCT24,166.92,0.00,67004.0162,94.87,0.0,0.0000,0.0470,...,-32.17943,0.67841,0.00003,0.98223,7249.60,2.2,0.0000,0.0490,0.0490,day
0,0.0,0.2,SYN.BTC-23OCT24,87.32,0.00,67004.0162,94.87,0.0,0.0000,0.0001,...,-11.99180,0.80508,0.00003,-0.02092,155.55,9.4,-75.0000,0.0001,0.0004,day
0,0.0,10.1,SYN.BTC-23OCT24,123.62,0.00,67004.0162,66.06,0.0,0.0000,0.0320,...,-28.12154,0.85137,0.00005,0.97682,76785.67,30.2,24.5902,0.0305,0.0390,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,24.0,10.7,SYN.BTC-24OCT24,61.34,52.60,67084.1131,61.31,0.0,0.0001,0.0003,...,-19.86881,2.09783,0.00002,0.02303,654.01,17.5,40.0000,0.0002,0.0007,day
0,5.4,35.1,SYN.BTC-24OCT24,91.56,0.00,67084.1131,61.31,0.0,0.0730,0.0760,...,-53.33456,2.07945,0.00002,-0.97716,17800.84,4.0,0.0000,0.0660,0.0660,day
0,19.8,5.5,SYN.BTC-24OCT24,70.67,60.93,67084.1131,65.76,0.0,0.0001,0.0003,...,-12.04746,1.36247,0.00001,0.01375,0.00,0.0,NaN,NaN,NaN,day
0,13.4,35.1,SYN.BTC-24OCT24,105.56,0.00,67084.1131,65.76,0.0,0.0880,0.0910,...,-35.29061,1.28271,0.00001,-0.98703,21838.94,4.0,0.0000,0.0810,0.0810,day


In [36]:
# print data and time for log
print('Date and time: ' +  datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S") + ' , format: dd/mm/yyyy hh:mm:ss')

# download data -- BTC and ETH Options
btc_data = get_option_data('BTC')
#eth_data = get_option_data('ETH')

# The data is now stored in Pandas DataFrames 'btc_data' and 'eth_data'
print('BTC data collected')
print(btc_data.head()) # Display first few rows of BTC data

#print('ETH data collected')
#print(eth_data.head()) # Display first few rows of ETH data

Date and time: 22/10/2024 23:14:20 , format: dd/mm/yyyy hh:mm:ss


c:\Users\Ethan Vaz Falcao\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\internals\concat.py:415: RuntimeWarning: coroutine 'get_option_data' was never awaited
  if not is_scalar(val) or not isna(val):
C:\Users\Ethan Vaz Falcao\AppData\Local\Temp\ipykernel_32604\768196887.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coin_df = pd.concat(coin_df)
 12%|█▏        | 116/948 [00:58<07:02,  1.97it/s]

BTC data collected
   best_bid_amount  best_ask_amount underlying_index  ask_iv  bid_iv  \
0              0.0             35.1  SYN.BTC-23OCT24  194.59    0.00   
0              7.8              0.0  SYN.BTC-23OCT24    0.00  112.08   
0              0.0             35.1  SYN.BTC-23OCT24  166.37    0.00   
0              0.0              0.2  SYN.BTC-23OCT24   89.19    0.00   
0              0.0             10.1  SYN.BTC-23OCT24  123.92    0.00   

   underlying_price  mark_iv  interest_rate  best_bid_price  best_ask_price  \
0        67079.1593   112.01            0.0          0.0000          0.0625   
0        67079.5121   112.03            0.0          0.0001          0.0000   
0        67079.5121    96.71            0.0          0.0000          0.0480   
0        67079.9531    96.72            0.0          0.0000          0.0001   
0        67079.9531    67.97            0.0          0.0000          0.0330   

   ...  greeks.theta  greeks.vega  greeks.gamma  greeks.delta  \
0  ...  

In [46]:
btc_data.to_csv('data/btc_data.csv', index=False)
#eth_data.to_csv('data/eth_data.csv', index=False)